In [34]:
from haystack.document_stores import FAISSDocumentStore,ElasticsearchDocumentStore
from haystack.nodes import DensePassageRetriever
from nlp_common.acts_reader import ActsReader
import regex
import random

In [24]:
from transformers import AutoTokenizer

AutoTokenizer.from_pretrained("enelpol/czywiesz-question")
AutoTokenizer.from_pretrained("enelpol/czywiesz-context")

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/472 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/886k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129 [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='enelpol/czywiesz-context', vocab_size=50000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'})

In [19]:
elastic_store = ElasticsearchDocumentStore()
faiss_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

In [32]:
faiss_retriever = DensePassageRetriever(
    document_store=faiss_store,
    query_embedding_model="enelpol/czywiesz-question",
    passage_embedding_model="enelpol/czywiesz-context",
)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'HerbertTokenizerFast'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

### Preprocess

In [38]:
reader = ActsReader("../ustawy")
bills = [ (name, text) for name, _, text in reader.all_acts() ]

def split_articles_single_bill(bill):
    r = regex.compile(r"Art\.[\s|\n]\d+")
    art_cleanup_re = regex.compile(r'\n*|\s*|\t*')
    new_line_re = regex.compile(r'\n+|\s+|\t+')
    
    expected_num = 1
    pos = []
    for m in r.finditer(bill):
        aid = art_cleanup_re.sub('', m.group(0))
        if aid[4:] == str(expected_num):
            expected_num += 1
            pos.append((aid, m.start(0)))

    if len(pos) < 1:
        return []
    
    arts = []
    for i in range(1, len(pos)):
        sub = bill[pos[i-1][1]:pos[i][1]]
        sub = new_line_re.sub(' ', sub)
        arts.append((pos[i-1][0],sub))
    
    sub = bill[pos[-1][1]:]
    sub = new_line_re.sub(' ', sub)
    arts.append((pos[-1][0],sub))
         
    return arts
    
def split_articles(bills):
    docs = []
    for name, bill in bills:
        arts = split_articles_single_bill(bill)
        for art,content in arts:
            docs.append({
                "content": content,
                "meta": { "name": f"{name}:{art}" }
            })
    return docs
        
articles = split_articles(bills)

In [8]:
elastic_store.write_documents(articles)

In [ ]:
# faiss_store.write_documents(articles)
# faiss_store.update_embeddings(faiss_retriever)

### Questions

In [46]:
random.choice(articles)

{'content': 'Art. 268. Właściwe organy Państwowej Straży Pożarnej w razie wystąpienia awarii przemysłowej są obowiązane do:  1) podjęcia działań operacyjno-ratowniczych we współpracy z prowadzącym  zakład,  2) zebrania informacji niezbędnych do dokonania analizy awarii i  sformułowania zaleceń dla prowadzącego zakład,  3) sprawdzenia, czy prowadzący zakład podjął wszystkie konieczne środki  zaradcze,  4) opracowania zaleceń dotyczących zastosowania w przyszłości określonych  środków zapobiegawczych,  5) sprawdzenia, czy prowadzący zakład wdrożył zalecenia właściwego organu  Państwowej Straży Pożarnej.  ',
 'meta': {'name': '2001_627.txt:Art.268'}}

In [ ]:
questions = [
    { "question": "Kto jest ubezpieczonymi na podstawie przepisów prawa?", "passage_id": "2001_906.txt:Art.2" },
    { "question": "Na czyj wniosek Rada Ministrów ustala cenę minimalną ziemniaków?", "passage_id": "2001_83.txt:Art.11" },
    { "question": "Do czego zobowiązane są organy Państwowej Straży Pożarnej w razie wystąpienia awarii przemysłowej?", 
      "passage_id": "2001_627.txt:Art.268" },
]